# Span Detection baseline check

## Prepare env

1. Import dependencies

In [ ]:
import random
import time

import numpy as np
import torch
import os

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
)

from src.util.torch_device import resolve_torch_device
from src.data.span_detection_ds import ManipulationDetectionDataset
from src.definitions import (
    RAW_DATA_FOLDER,
    PROCESSED_DATA_FOLDER,
)
from src.model.span_detection_metrics import compute_metrics_baseline
from src.model.baseline import (
    AllZerosModel,
    AllOnesModel,
    NormalDistModel,
    UniformDistModel,
)

2. Prepare Env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

model_checkpoint = "EvanD/xlm-roberta-base-ukrainian-ner-ukrner"

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

3. Load dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset_blueprint = ManipulationDetectionDataset(
    tokenizer=tokenizer,
    raw_path=RAW_DATA_FOLDER / "span-detection.parquet",
    processed_path=PROCESSED_DATA_FOLDER / "span-detection",
    seed=random_seed,
)

dataset = dataset_blueprint.read()
dataset = dataset.remove_columns(["id", "content"])

data_collator = DataCollatorForTokenClassification(tokenizer)

dataset_slice = dataset["test"][:]
features = [dict(zip(dataset_slice, t)) for t in zip(*dataset_slice.values())]
padded_dataset = data_collator(features)

y_true = padded_dataset["labels"]

evaluator = compute_metrics_baseline(dataset_blueprint)

## Check model that returns only 0s

1. Prepare model

In [4]:
model = AllZerosModel()

2. Eval model

In [5]:
y_pred = model(padded_dataset["input_ids"])

evaluation_feedback = evaluator((y_pred, y_true))

evaluation_feedback

/Users/alexandermelashchenko/Workspace/unlp-2025-manipulation-detector/.venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': np.float64(0.0),
 'recall': np.float64(0.0),
 'f1': np.float64(0.0),
 'accuracy': 0.7423167195913296}

## Check model that returns only 1s

1. Prepare model

In [6]:
model = AllOnesModel()

2. Eval model

In [7]:
y_pred = model(padded_dataset["input_ids"])

evaluation_feedback = evaluator((y_pred, y_true))

evaluation_feedback

{'precision': np.float64(0.013054830287206266),
 'recall': np.float64(0.0063532401524777635),
 'f1': np.float64(0.008547008547008546),
 'accuracy': 0.25768328040867045}

## Check model that samples from uniform distribution

1. Prepare model

In [8]:
model = UniformDistModel()

2. Eval model

In [9]:
y_pred = model(padded_dataset["input_ids"])

evaluation_feedback = evaluator((y_pred, y_true))

evaluation_feedback

{'precision': np.float64(0.0020844761382336806),
 'recall': np.float64(0.0241423125794155),
 'f1': np.float64(0.0038376085639264795),
 'accuracy': 0.5009250310644761}

## Check model that samples from uniform distribution

1. Prepare model

In [10]:
model = NormalDistModel()

2. Eval model

In [11]:
y_pred = model(padded_dataset["input_ids"])

evaluation_feedback = evaluator((y_pred, y_true))

evaluation_feedback

{'precision': np.float64(0.0024009603841536613),
 'recall': np.float64(0.02795425667090216),
 'f1': np.float64(0.004422110552763819),
 'accuracy': 0.501477288416402}